In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from IPython.display import clear_output
from src.configs import *
from src.plotting import *
from src.pcs import *

# Compute and save gridded CRH

In [50]:
q_ds = xr.open_dataset(data_dir('era5.merged.daily_q.nc')).Q.sel(level=np.linspace(100, 1000, 10))
r_ds = xr.open_dataset(data_dir('era5.merged.daily_r.nc')).R.sel(level=np.linspace(100, 1000, 10))
qs = q_ds.copy(data=q_ds.data/r_ds.data).rename('sat_specific_humidity')
crh = q_ds.integrate('level')/qs.integrate('level')
crh.to_netcdf(data_dir('era5.merged.daily_crh.nc'))

PermissionError: [Errno 13] Permission denied: b'/Users/pedro/extreme_precipitation_in_gpm/data/era5.merged.daily_crh.nc'

# Compute local mean CRH around PFs

In [8]:
region = 'CIO'
radius_in_degs = 2

In [9]:
# load PFs, look only in tropical region
pf_df = load_pf_stats(region)
pf_df = pf_df.where(pf_df.mean_latitude.abs() < 10).dropna()
pc_df = load_pf_stats(region)
pf_lat = pf_df.mean_latitude.values
pf_lon = pf_df.mean_longitude.values
pf_time = pd.to_datetime(pf_df.observation_time, format='%Y%m%d_%H:%M:%S')
pf_day = pf_time.dt.floor('D').values

In [10]:
# load CRH
crh = xr.open_dataarray(data_dir('era5.merged.daily_crh.nc'))
crh['time'] = crh['time'].dt.floor('D')
crh['lon'] = crh.lon - 180

In [11]:
def lat_lon_grid_slice(lat0, lon0, time0, radius_in_degs=5):
    slice_dict = {
        'lat': slice(lat0-radius_in_degs/2, lat0+radius_in_degs/2),
        'lon': slice(lon0-radius_in_degs/2, lon0+radius_in_degs/2),
        'time': time0
    }
    return slice_dict
grid_slices = [lat_lon_grid_slice(lat0, lon0, time0, radius_in_degs=5) for (lat0, lon0, time0) in zip(pf_lat, pf_lon, pf_day)]

In [12]:
crh_database_dict = {'crh': []}
for i, gs in enumerate(grid_slices):
    clear_output()
    print(f'PF {i+1} of {len(grid_slices)} ')
    obs_crh = crh.sel(gs)
    assert(obs_crh.lat.size>0 and obs_crh.lon.size>0)
    crh_database_dict['crh'].append(obs_crh.mean().item())
crh_database = pd.DataFrame.from_dict(crh_database_dict)

PF 49817 of 50539 


KeyboardInterrupt: 

In [48]:
pf_df.join(crh_database).to_csv(data_dir(f'{region}.tropical_pf_crh_data_{radius_in_degs}deg.csv'))